Assemble and display a tract

In [1]:
import collections
import numpy as np

import lsst.daf.persistence as dafPersist
import lsst.afw.image as afwImage
import lsst.afw.detection as afwDetect
import lsst.afw.geom as afwGeom
import lsst.afw.cameraGeom.utils as cameraGeomUtils
import lsst.afw.display.ds9 as ds9
import lsst.afw.display.utils as displayUtils
import lsst.afw.display.rgb as afwRgb

In [2]:
import os
os.environ["SUPRIME_DATA_DIR"] = "/tigress/HSC/HSC" if False else "/data1b/work/yasuda/mbrown"
rerun = "yasuda/5166"
dataPath = os.path.join(os.environ["SUPRIME_DATA_DIR"], "rerun", rerun)

butler = dafPersist.Butler(dataPath)
skymap = butler.get("deepCoadd_skyMap")

In [111]:
def assembleTiles(images):
    """Assemble a list of tiles according to their XY0 values"""
    bigBBox = afwGeom.BoxI()

    for im in images:
        bigBBox.include(im.getBBox(afwImage.PARENT))

    bigIm = afwImage.MaskedImageF(bigBBox)
    for im in images:
        if True:
            sub = bigIm.Factory(bigIm, im.getBBox(afwImage.PARENT), afwImage.PARENT)
            sub <<= im.getMaskedImage()
            del sub
        else:
            bigIm[im.getBBox(afwImage.PARENT)] = im

    return bigIm

#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-

def assemblePatches(patches, patchInfo):
    """Assemble a list of tiles according to their XY0 values"""
    bigBBox = afwGeom.BoxI()
    
    for p in patches:
        pi = patchInfo[p]
        bigBBox.include(pi.getInnerBBox())

    bigIm = afwImage.MaskedImageF(bigBBox)
    for p in patches:
        im = patches[p].getMaskedImage()
        pi = patchInfo[p]

        if True:
            sub = bigIm.Factory(bigIm, pi.getInnerBBox(), afwImage.PARENT)
            sub <<= im.Factory(im, pi.getInnerBBox(), afwImage.PARENT)
            del sub
        else:
            bigIm[pi.getInnerBBox()] = im[pi.getInnerBBox()]

    return bigIm

#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-

def getTract(butler, dataId, pps=None, butlerType="deepCoadd_calexp"):
    skymap = butler.get("deepCoadd_skyMap")
    patchInfo = dict(('%d,%d' % _.getIndex(), _) for _ in skymap[dataId["tract"]])

    if pps is None:
        pps = list(patchInfo)
        
    filterName = dataId["filter"]
    if len(filterName) == 1:
        filterName = "HSC-%s" % filterName.upper()
        
    patches = {}
    for pp in pps:
        patches[pp] = butler.get(butlerType, dataId, filter=filterName, patch=pp)

    return assemblePatches(patches, patchInfo)

def getPatchInfo(butler, dataId, pps=None, butlerType="deepCoadd_calexp"):
    skymap = butler.get("deepCoadd_skyMap")
    patchInfo = dict(('%d,%d' % _.getIndex(), _) for _ in skymap[dataId["tract"]])
    
    if pps is None:
        pps = list(patchInfo)

    filterName = dataId["filter"]
    if len(filterName) == 1:
        filterName = "HSC-%s" % filterName.upper()
        
    info = collections.OrderedDict()
    for pp in pps:
        patch = butler.get(butlerType, dataId, filter=filterName, patch=pp)
        info[pp] = patch.getInfo()

    return info

In [112]:
if True:
    info = getPatchInfo(butler, dataId=dict(tract=tract, filter='g'), pps=pps)

for pp in info:
    print pp
    inputs = info[pp].getCoaddInputs()
    w = 0
    for v in inputs.visits:
        w += v["weight"]
    for v in inputs.visits:
        print "\t%5d  %.3f" % (v.getId(), v["weight"]/w)

3,3
	24164  0.371
	24196  0.364
	24228  0.265
3,4
	24164  0.370
	24196  0.363
	24228  0.267
3,5
	24164  0.369
	24196  0.364
	24228  0.268
4,3
	24164  0.371
	24196  0.363
	24228  0.266
4,4
	24164  0.370
	24196  0.361
	24228  0.269
4,5
	24164  0.371
	24196  0.361
	24228  0.269
5,3
	24164  0.369
	24196  0.363
	24228  0.268
5,4
	24164  0.369
	24196  0.364
	24228  0.266
5,5
	24164  0.370
	24196  0.362
	24228  0.268


In [75]:
if False:
    p = butler.get('deepCoadd_calexp', tract=0, patch='5,5', filter='HSC-G', immediate=True)
    ds9.mtv(p)

In [76]:
if False:
    d = butler.get('deepCoadd_tempExp_diff', tract=0, patch='5,5', filter='HSC-G', visit=24164, immediate=True)
    ds9.mtv(d, frame=1)

In [48]:
tract = 0
pps = ["%d,%d" % (i , j) for i in range(3, 6) for j in range(3, 6)]
#pps = ["%d,%d" % (i , j) for i in range(5, 6) for j in range(5, 6)]
visits = [24164, 24196, 24228]

fullPatchIm = getTract(butler, dataId=dict(tract=tract, filter='g'), pps=pps,
                       butlerType="deepCoadd_calexp")
patchIm = fullPatchIm

ds9.mtv(patchIm, frame=1, title="%d %s" % (tract, str(pps)[1:-1].replace("'", "")))

In [50]:
diffIms = collections.OrderedDict()
for frame, v in enumerate(visits, 2):
    if v in diffIms:
        continue
    print v
    diffIms[v] = getTract(butler, dataId=dict(tract=tract, filter='g', visit=v), pps=pps,
                          butlerType="deepCoadd_tempExp_diff")
    if False:
        ds9.mtv(diffIms[v], frame=frame, title=v)

24164
24196
24228


In [51]:
visitIms = collections.OrderedDict()
for frame, v in enumerate(diffIms.keys(), 2 + 3):
    if v in visitIms:
        continue
    print v
    visitIms[v] = getTract(butler, dataId=dict(tract=tract, visit=v, filter='g'), pps=pps,
                           butlerType="deepCoadd_tempExp")
    if False:
        ds9.mtv(visitIms[v], frame=frame, title=v)

24164
24196
24228


Now some code to mask the artefacts in a difference image

In [74]:
if False:
    fullPatchIm = patchIm

if True:
    bbox = fullPatchIm.getBBox()
else:
    bbox = afwGeom.BoxI(afwGeom.PointI(9000, 9000), afwGeom.PointI(12000 - 1, 12000 - 1))

patchIm = fullPatchIm[bbox]

if False:
    ds9.mtv(patchIm, frame=1, title="%d %s" % (tract, str(pps)[1:-1].replace("'", "")))

threshold = 2    # sigma (but the data's correlated, so this isn't directly interpretable)
nGrow = 15       # pixels; saturated footprints only
fs = afwDetect.FootprintSet(patchIm, afwDetect.Threshold(threshold, afwDetect.Threshold.PIXEL_STDEV))

frame = 1
x0, y0 = patchIm.getXY0()
mask = patchIm.getMask()
image = patchIm.getImage()
brightThreshold = np.inf
clippedNpixMin = 1000

CLIPPED = 1 << mask.getMaskPlane("CLIPPED")
SAT = 1 << mask.getMaskPlane("SAT")
INTRP = 1 << mask.getMaskPlane("INTRP")

maskedPixels = (patchIm.getMask().getArray() & SAT)
height, width = maskedPixels.shape

for foot in fs.getFootprints():
    peak = foot.getPeaks()[0]
    x, y, = peak.getIx(), peak.getIy()
    setBits = foot.overlapsMask(mask)
    if image.get(x - x0, y - y0) > brightThreshold or (setBits & SAT) or \
       ((setBits & CLIPPED) and foot.getNpix() >= clippedNpixMin):
        if (setBits & SAT) and nGrow:
            foot = afwDetect.growFootprint(foot, nGrow)
        for s in foot.getSpans():
            sy = s.getY() - y0
            sx0, sx1 = s.getX0() - x0, s.getX1() - x0
            
            try:
                maskedPixels[sy, sx0:sx1+1] = 1
            except IndexError:
                if sy < 0 or sy >= height:
                    continue
                if sx0 < 0:
                    sx0 = 0
                if sx1 >= width:
                    sx1 = width - 1
                maskedPixels[sy, sx0:sx1+1] = 1

if True:             # include union of chip gaps in maskedPixels
    for v in diffIms:
        maskedPixels[np.isnan(diffIms[v][bbox].getImage().getArray())] = 1

cleaned = collections.OrderedDict()
for frame, v in enumerate(diffIms, 2):
    cleaned[v] = diffIms[v][bbox].clone()
    imArr = cleaned[v].getImage().getArray()
    mskArr = cleaned[v].getMask().getArray()
    mskArr &= CLIPPED

    bad = np.logical_or(np.isnan(imArr), maskedPixels > 0)        
    good = np.logical_and(np.isfinite(imArr), maskedPixels == 0)

    q25, q50, q75 = np.percentile(imArr[good], [25, 50, 75])
    imArr[bad] = np.random.normal(q50, 0.741*(q75 - q25), size=np.sum(bad))
    mskArr[bad] |= INTRP
    
    if False:
        ds9.mtv(cleaned[v], frame=frame, title=v)
    if False:
        ds9.mtv(visitIms[v][bbox], frame=frame + 3, title=v)

In [76]:
R, G, B = [im.getImage().clone() for im in cleaned.values()]

# make the blue a bit brighter by mixing in some green
G.getArray()[:] += 0.5*B.getArray()

# Set the pixel masks
mskVal = 5
for im in [G, B]:
    im.getArray()[bad] += mskVal
    
clippedPixels = (patchIm.getMask().getArray() & CLIPPED) == CLIPPED
for im in [R, B]:
    im.getArray()[clippedPixels] += mskVal

#
# Add in the ghost of the direct image
#
addGhost = True
if addGhost:
    for im in [R, G, B]:
        im.getArray()[:] += 100*patchIm.getImage().getArray()
    
rgb = afwRgb.makeRGB(R, G, B, minimum=-10, range=30, Q=8)
afwRgb.writeRGB("tract-0-3,5-diff%s.png" % ("-ghost" if addGhost else ""), rgb)

if False:
    afwRgb.displayRGB(rgb)

In [77]:
gray = afwRgb.makeRGB(*3*[patchIm[bbox]], minimum=-0.1, range=0.3, Q=8)
afwRgb.writeRGB("tract-0-3,5.png", gray)

In [59]:
if False:
    ds9.mtv(butler.get("deepCoadd_tempExp", tract=0, patch='3,3', filter='HSC-G', visit=24228))

In [113]:
calexp = butler.get("calexp", ccd=55, filter='HSC-G', visit=24228)
calib = calexp.getCalib()
print calib.getExptime()

180.0


In [132]:
import lsst.afw.cameraGeom as afwCamGeom
boreSight = calexp.getDetector().getCenter()   # pixels in FPA coordinates
pointing = calexp.getWcs().pixelToSky(boreSight.getPixels(1)) # where the boresight's pointing
print [_.asDegrees() for _ in pointing]

[195.5876425659588, 4.032620171976911]
